<a href="https://colab.research.google.com/github/kumiori/mec647/blob/main/test_ayoub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import site
import sys


try : 
    from google.colab import files
except:
    pass
    
try:
    from fenics import *; from mshr import *
except ImportError as e:
    !apt-get install software-properties-common
    !add-apt-repository -y ppa:fenics-packages/fenics
    !apt-get update -qq
    !apt install -y fenics    
    site.addsitedir('/usr/lib/petsc/lib/python3/dist-packages')  # Always appends to end

    from fenics import *; from mshr import *
    from ufl import replace

!fenics-version

2019.2.0.dev0


In [24]:
import matplotlib.pyplot as plt;
import numpy as np;
import math  
from IPython.display import clear_output, display, update_display; import time; 
# import dolfin.common.plotting as fenicsplot 
import time
%matplotlib inline
plt.rcParams['image.cmap'] = 'viridis'

In [30]:
L = 1.; H = .5
a=.1
P1= Point ((L/2-a),0)
P2= Point (L/2,a)
P3= Point ((L/2+a),0)
geom_1= (P1,P2,P3)
cell_size = 0.02
nel = int(L/cell_size)
b= Point ((L/2-a),a)
geom= Rectangle(Point(0., 0.), Point(L, H))
mesh = generate_mesh(geom, nel)
plot(mesh)

NameError: ignored